In [2]:
import pandas as pd
import requests
from datetime import datetime, timedelta

In [3]:
df_attacks = pd.read_csv('/mnt/d/Adalab/ejercicios-de-adalab/modulo-2/8-ETL-mod-2/files/attacks_limpieza_completa.csv', index_col = 0)
df_etl2 = pd.read_csv('/mnt/d/Adalab/ejercicios-de-adalab/modulo-2/8-ETL-mod-2/files/etl2.csv', index_col = 0)
df_etl1 = pd.read_csv('/mnt/d/Adalab/ejercicios-de-adalab/modulo-2/8-ETL-mod-2/files/df_ETL1_clima.csv')


In [51]:
class Extraccion: 
    
    def __init__(self, lat, long, pais):
        self.pais = pais
        self.lat = lat
        self.long = long
        
    def llamada_API(self, producto):
        self.producto = producto
          
        url = f'http://www.7timer.info/bin/api.pl?lon=-{self.long}&lat={self.lat}&product=meteo&output=json'
        response = requests.get(url=url)
        codigo_estado = response.status_code
        razon_estado = response.reason
        if codigo_estado == 200:
                print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,
                  'y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 402:
            print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,
                  'y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 404:
            print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:',
                  codigo_estado,' y como razón del código de estado: ',razon_estado)
        else:
            print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,
                  'y como razón del código de estado: ',razon_estado) 
        
        df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
        return df
    
    def limpiar_civil(self, df): 

        hoy = datetime.now()
        hoy = datetime.strftime(hoy, '%Y-%m-%d')

        # creamos la nueva columna
        df["fecha"] = hoy

        # de nuevo usamos el self para crear las nuevas columnas en función de los parámetros pasados al definir 
        # la clase. Recordamos, el caminito de baldosas amarillas.
        df["latitud"] = self.lat
        df["longitud"] = self.lon
        df["pais"] = self.pais
        return df

    def limpiar_meteo(self, df):
    
        #seleccionamos solo las columnas que nos interesan
        df = df[["timepoint", "cloudcover", "highcloud"]]

        # creamos la columna de fecha: 
        hoy = datetime.now()
        hoy = datetime.strftime(hoy, '%Y-%m-%d')
        df["fecha"] = hoy


        # insertamos las columnas de la localidad
    
        df["pais"] = self.pais

        return df
    
    def juntar_dfs(self, df_attacks, df_etl2): 

        df_hoy = pd.merge(df_attacks, df_etl2 , on=['fecha', "timepoint", "pais"], how = "right")
        print("El df de hoy es ", df_hoy.columns)
        print("-----------------------------------------")
        print("El df de completo es ", df_completo.columns)
        # lo primero que hacemos es concatenar los dataframes con la información general. df_completo y df_civil
        df_completo = pd.concat([df_completo, df_hoy], axis = 0)

        
        # guardamos los datos
        df_completo.to_pickle('datos_actualizados_3.pkl')
        df_completo.to_csv('datos_actualizados_3.csv')

        return df_completo
    
    def chequear_datos(self, df): 
    
        print("Las columnas son:", "\n")
        print(list(df.columns))
        print("-----------------------------------------")

        print("Los tipos de datos que tenemos son:", "\n")
        print(df.dtypes)
        print("-----------------------------------------")

        print("El porcentaje de nulos:", "\n")
        print((df.isnull().sum() / df.shape[0]) *  100)

    def limpiar_dataframe(self, df, lista_columnas = []): 
        
        
     #convertimos la fecha a datetime
        df["fecha"] = pd.to_datetime(df["fecha"])

        # reemplazamos los nulos de las columnas por la media
         # lista de columnas en las que queremos reemplazar los nulos
        # df[lista_columnas]=df[lista_columnas].fillna(df.mean())

        
        # renombrar columnas
        
        # df.rename(columns = {"ciudad_x": "ciudad"}, inplace = True)

        # quitar % 
        df["rh2m"] = df["rh2m"].replace(r"%", "", regex = True)

        # guardamos los datos una vez limpios
        df.to_pickle('../files/datos_actualizados.pkl')
        df.to_csv('../files/datos_actualizados.csv')

        return df


    


In [5]:
clase_extraccion = Extraccion(39.7837304, -100.445882,'usa')
    

In [6]:
df_meteo_usa = clase_extraccion.llamada_API('meteo')

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200 y como razón del código de estado:  OK


In [ ]:
df_meteo_usa = clase_extraccion.limpiar_meteo(df_meteo_usa)

In [8]:
df_meteo_usa.head(2)

,timepoint,cloudcover,highcloud,fecha,pais
0,3,7,-9999,2023-01-11,usa
1,6,8,-9999,2023-01-11,usa


In [9]:
clase_extraccion = Extraccion(-24.7761086, 134.755,'australia')

In [17]:
df_meteo_australia = clase_extraccion.llamada_API('meteo')

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200 y como razón del código de estado:  OK


In [ ]:
df_meteo_australia = clase_extraccion.limpiar_meteo(df_meteo_australia)

In [19]:
df_meteo_australia.head(1)

,timepoint,cloudcover,highcloud,fecha,pais
0,3,2,-9999,2023-01-11,australia


In [28]:
clase_extraccion = Extraccion(-28.8166236, 24.991639,'south africa')

In [29]:
df_meteo_southafrica = clase_extraccion.llamada_API('meteo')

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200 y como razón del código de estado:  OK


In [ ]:
df_meteo_southafrica = clase_extraccion.limpiar_meteo(df_meteo_southafrica)

In [31]:
df_meteo_southafrica.head(1)

,timepoint,cloudcover,highcloud,fecha,pais
0,3,6,-9999,2023-01-11,south africa


In [33]:
clase_extraccion = Extraccion(5.6816069, -144.2489081,'papua new guinea')

In [36]:
df_meteo_newguinea = clase_extraccion.llamada_API('meteo')

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200 y como razón del código de estado:  OK


In [ ]:
df_meteo_newguinea = clase_extraccion.limpiar_meteo(df_meteo_newguinea)

In [38]:
df_meteo_newguinea.head(1)

,timepoint,cloudcover,highcloud,fecha,pais
0,3,1,-9999,2023-01-11,papua new guinea


In [41]:
clase_extraccion = Extraccion(-41.5000831, 144.2489081,'new zealand')

In [39]:
df_meteo_newzealand = clase_extraccion.llamada_API('meteo')

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200 y como razón del código de estado:  OK


In [ ]:
df_meteo_newzealand = clase_extraccion.limpiar_meteo(df_meteo_newzealand)

In [44]:
df_meteo_newzealand.head(2)

,timepoint,cloudcover,highcloud,fecha,pais
0,3,1,-9999,2023-01-11,new zealand
1,6,1,-9999,2023-01-11,new zealand


In [54]:
# UNIR DF'S
def_meteo_attacks = clase_extraccion.juntar_dfs(df_attacks, df_etl2)
def_meteo_attacks.head(2)

KeyError: 'fecha'